# Simulation

[[1]](https://github.com/eitcom/pyEIT)

In [11]:
# coding: utf-8
""" demo on forward 2D """
# Copyright (c) Benyuan Liu. All Rights Reserved.
# Distributed under the (new) BSD License. See LICENSE.txt for more info.
from __future__ import absolute_import, division, print_function

import matplotlib.pyplot as plt
import numpy as np
import pyeit.eit.protocol as protocol
import pyeit.mesh as mesh
from pyeit.eit.fem import EITForward, Forward
from pyeit.eit.interp2d import pdegrad, sim2pts
from pyeit.mesh.shape import thorax
from pyeit.mesh.wrapper import PyEITAnomaly_Circle
from PIL import Image
import os

from src.util import plot_mesh

In [12]:
#helper = []
#helper.append()

#helper2 = np.array()


In [13]:
""" 0. build mesh """

n_el = 16  # nb of electrodes
use_customize_shape = False
if use_customize_shape:
    # Mesh shape is specified with fd parameter in the instantiation, e.g : fd=thorax
    mesh_obj = mesh.create(n_el, h0=0.05, fd=thorax)
else:
    mesh_obj = mesh.create(n_el, h0=0.05)
el_pos = mesh_obj.el_pos

# extract node, element, alpha
pts = mesh_obj.node
tri = mesh_obj.element
x, y = pts[:, 0], pts[:, 1]
mesh_obj.print_stats()
protocol_obj = protocol.create(n_el, dist_exc=4, step_meas=1, parser_meas="std")

#def calculateData (mesh):
 #   protocol_obj = protocol.create(n_el, dist_exc=4, step_meas=1, parser_meas="std")
#    for i in range(len(protocol_obj.ex_mat)):
 #       ex_line = protocol_obj.ex_mat[15].ravel() 
 #       fwd = Forward(mesh)
 #       f = fwd.solve(ex_line)
 #       fwd = EITForward(mesh_obj, protocol_obj)
 #       f = np.real(f)
  #      v = fwd.solve_eit(perm=mesh.perm)   #berechnet den Potentialvektor (192 Werte)
  #  return (f, v)

def calculateData (mesh):
    ex_line = protocol_obj.ex_mat[3].ravel() 
    fwd = Forward(mesh)
    f = fwd.solve(ex_line)
    fwd = EITForward(mesh_obj, protocol_obj)
    f= np.real(f)
    v = fwd.solve_eit(perm=mesh.perm)   #berechnet den Potentialvektor (192 Werte)
    return (f, v)

# Erstellt die einzelnen Positionen der Anomalie (abhängig von der Anzahl "Nsteps"), Größe der Anomalie über r definierbar, Radius der Kreisbahn mit r_path einstellbar 
def createAnomaly (Nsteps,r,r_path):
    anglePos = np.linspace(0, 2*np.pi, Nsteps)           # P(cos(alpha)=x, sin(alpha)=y)
    perm=100.0
    mesh_new_list = []
    for a in anglePos:
        anomaly = PyEITAnomaly_Circle(center=[np.cos(a)*r_path,np.sin(a)*r_path], r=r, perm=perm) #erstellt Anomalie an den einzelnen Positionen/Winkeln
        anomaly_mesh = mesh.set_perm(mesh_obj, anomaly=anomaly, background=1.0)
        f, v = calculateData(anomaly_mesh)
        mesh_dict = {"mesh": anomaly_mesh, "x":np.cos(a)*r_path, "y":np.sin(a)*r_path, "radius":r, "perm_init":perm, "f":f, "v":v, "anomaly": anomaly}
        mesh_new_list.append(mesh_dict) #erstellt eine Liste aus den Mesh-Dict Daten
        #perm = mesh_new.perm    
    return mesh_new_list


2D mesh status:
1476 nodes, 2821 elements


**Erstellt Gif-Animation**

In [ ]:
def createAnimation(mesh_new_list, protocol_obj, output_gif="animation_with_movement.gif"):
    
    image_files = []

    # Loop durch jede Anomalie-Position
    for i, mesh_data in enumerate(mesh_new_list):

        # Loop durch jedes Elektrodenpaar für die aktuelle Anomalie-Position
        for j, ex_line in enumerate(protocol_obj.ex_mat):
            fig, ax1 = plt.subplots(figsize=(9, 6))

            # Potentialverteilung für dieses Elektrodenpaar
            fwd = Forward(mesh_data["mesh"])
            f = np.real(fwd.solve(ex_line.ravel()))

            # Äquipotentiallinien basierend auf der Einspeisung
            vf = np.linspace(min(f), max(f), 64)
            ax1.tricontour(x, y, tri, f, vf, cmap=plt.cm.viridis)

            ax1.tripcolor(
                x,
                y,
                tri,
                np.real(mesh_data["mesh"].perm),
                edgecolors="k",
                shading="flat",
                alpha=0.5,
                cmap=plt.cm.Greys,
            )

            # plottet Elektroden
            ax1.plot(x[el_pos], y[el_pos], "ro")
            for e_idx, e in enumerate(el_pos):
                ax1.text(x[e], y[e], str(e_idx + 1), size=12)

            ax1.set_title(f"Equi-potential lines (Anomaly Position {i+1}, Electrodes {j+1})")
            ax1.set_aspect("equal")
            ax1.set_ylim([-1.2, 1.2])
            ax1.set_xlim([-1.2, 1.2])
            fig.set_size_inches(6, 6)

            # Speichert das Bild
            filename = f"frame_{i}_{j}.png"
            plt.savefig(filename)
            image_files.append(filename)
            plt.close(fig)

    # Erstellt das GIF
    frames = [Image.open(image) for image in image_files]
    frames[0].save(output_gif, format="GIF", append_images=frames[1:], save_all=True, duration=100, loop=0)

    # Einzelbilder löschen
    for image in image_files:
        os.remove(image)

def storeData (mesh_new_list):
    for i in range(len(mesh_new_list)):
    
        np.savez("data/sample_{0:06d}.npz".format(i), v=mesh_new_list[i]["v"], anomalie=mesh_new_list[i]["anomaly"])

        
mesh_new_list = createAnomaly(100, 0.2, 0.5)  
protocol_obj = protocol.create(n_el, dist_exc=4, step_meas=1, parser_meas="std")
#createAnimation(mesh_new_list, protocol_obj)
storeData(mesh_new_list)

**load data**

In [15]:
from glob import glob

In [16]:
file_list = np.sort(glob("data/*.npz"))

In [17]:
V = list()  # voltages
A = list()  # anomalies
for file in file_list:
    tmp = np.load(file, allow_pickle=True)
    V.append(tmp["v"])
    A.append(tmp["anomalie"].tolist())

In [18]:
A

[PyEITAnomaly_Circle(center=array([0.5, 0. ]), perm=100.0, r=0.2),
 PyEITAnomaly_Circle(center=array([0.49899334, 0.03171196]), perm=100.0, r=0.2),
 PyEITAnomaly_Circle(center=array([0.49597741, 0.06329623]), perm=100.0, r=0.2),
 PyEITAnomaly_Circle(center=array([0.49096435, 0.09462562]), perm=100.0, r=0.2),
 PyEITAnomaly_Circle(center=array([0.48397435, 0.12557399]), perm=100.0, r=0.2),
 PyEITAnomaly_Circle(center=array([0.47503556, 0.15601672]), perm=100.0, r=0.2),
 PyEITAnomaly_Circle(center=array([0.46418397, 0.18583123]), perm=100.0, r=0.2),
 PyEITAnomaly_Circle(center=array([0.45146327, 0.21489746]), perm=100.0, r=0.2),
 PyEITAnomaly_Circle(center=array([0.43692469, 0.24309837]), perm=100.0, r=0.2),
 PyEITAnomaly_Circle(center=array([0.42062677, 0.27032041]), perm=100.0, r=0.2),
 PyEITAnomaly_Circle(center=array([0.40263513, 0.29645396]), perm=100.0, r=0.2),
 PyEITAnomaly_Circle(center=array([0.38302222, 0.3213938 ]), perm=100.0, r=0.2),
 PyEITAnomaly_Circle(center=array([0.36186

In [19]:
A[3]

PyEITAnomaly_Circle(center=array([0.49096435, 0.09462562]), perm=100.0, r=0.2)

In [20]:
A[3].r

0.2